# Running ETL to Build the Document Corpus


This notebook walks through the process for setting up the corpus of Full Stack documents that the bot searches over.

In each case, we have to

- Extract data from its natural habitat, like YouTube or GitHub
- Transform it into a format that is useful for our purposes
- Load it into our database in that format

hence the acronym "ETL".


In [ ]:
%env ENV=dev
!make secrets  # you'll need credentials for Mongo and Modal to run this
db, collection = "fsdl-dev", "ask-fsdl"  # we run this in a dev database

# drop the collection if it exists, it's just dev
!modal run app.main::drop_docs --db {db} --collection {collection}


In [ ]:
import json
from pathlib import Path
import pprint

from etl import markdown, pdfs, shared, videos
from etl.shared import display_modal_image

pp = pprint.PrettyPrinter(indent=2)


## PDFs: arXiV Papers


```bash
!modal run etl/pdfs.py --json-path data/llm-papers.json
```


In [ ]:
display_modal_image(shared.image)


In [ ]:
display_modal_image(pdfs.image)


In [ ]:
papers_path = Path("data") / "llm-papers.json"

with open(papers_path) as f:
    pdf_infos = json.load(f)

pdf_infos[:100:20]


In [ ]:
with pdfs.stub.run():
    # first, we enrich the paper data by finding direct PDF URLs where we can
    paper_data = pdfs.get_pdf_url.map(
        pdf_infos[::25], return_exceptions=True  # subsampling to run faster
    )
    # then we turn the PDFs into JSON documents
    documents = shared.unchunk(  # each pdf creates a list of documents, one per page, so we flatten
        # after we run the extract_pdf function on Modal to get those pages
        pdfs.extract_pdf.map(paper_data, return_exceptions=True)
    )


In [ ]:
pp.pprint(documents[0]["metadata"])


In [ ]:
with shared.stub.run():
    # we split our document list into 10 pieces, so that we don't open too many connections
    chunked_documents = shared.chunk_into(documents, 10)
    list(
        shared.add_to_document_db.map(
            chunked_documents, kwargs={"db": db, "collection": collection}
        )
    )


In [ ]:
with shared.stub.run():
    # pull only arxiv papers
    query = {"metadata.source": {"$regex": "arxiv\.org", "$options": "i"}}
    # project out the text field, it can get large
    projection = {"text": 0}
    # get just one result to show it worked
    result = shared.query_one.remote(query, projection, db=db, collection=collection)

pp.pprint(result)


## Markdown Files: Lectures


```bash
!modal run etl/markdown.py --json-path data/lectures-2022.json
```


In [ ]:
display_modal_image(markdown.image)


In [ ]:
markdown_path = Path("data") / "lectures-2022.json"

with open(markdown_path) as f:
    markdown_corpus = json.load(f)

website_url, md_url = (
    markdown_corpus["website_url_base"],
    markdown_corpus["md_url_base"],
)

lectures = markdown_corpus["lectures"]

lectures[0]


In [ ]:
with markdown.stub.run():
    documents = shared.unchunk(  # each lecture creates multiple documents, one per section so we flatten
        markdown.to_documents.map(
            lectures,
            kwargs={"website_url": website_url, "md_url": md_url},
            return_exceptions=True,
        )
    )


In [ ]:
pp.pprint(documents[1]["metadata"])


In [ ]:
from IPython.display import IFrame

IFrame(src=documents[1]["metadata"]["source"], width=800, height=400)


In [ ]:
with shared.stub.run():
    chunked_documents = shared.chunk_into(documents, 10)
    list(
        shared.add_to_document_db.map(
            chunked_documents, kwargs={"db": db, "collection": collection}
        )
    )


In [ ]:
with shared.stub.run():
    # pull only lectures
    query = {"metadata.source": {"$regex": "lecture", "$options": "i"}}
    # project out the text field, it can get large
    projection = {"text": 0}
    # get just one result to show it worked
    result = shared.query_one.remote(query, projection, db=db, collection=collection)

pp.pprint(result)


## Videos: YouTube Transcripts


In [ ]:
display_modal_image(videos.image)


In [ ]:
videos_path = Path("data") / "videos.json"

with open(videos_path) as f:
    video_infos = json.load(f)

video_ids = [video["id"] for video in video_infos]

video_infos[0]


In [ ]:
with videos.stub.run():
    documents = shared.unchunk(  # each lecture creates multiple documents, one per chapter, so we flatten
        videos.extract_subtitles.map(
            video_infos[-3:],  # subsampling to run faster
            return_exceptions=True,
        )
    )


In [ ]:
pp.pprint(documents[1]["metadata"])


In [ ]:
from IPython.display import YouTubeVideo

id_str, time_str = documents[1]["metadata"]["source"].split("?v=")[-1].split("&t=")
YouTubeVideo(id_str, start=int(time_str.strip("s")), width=800, height=400)


In [ ]:
with shared.stub.run():
    chunked_documents = shared.chunk_into(documents, 10)
    list(
        shared.add_to_document_db.map(
            chunked_documents, kwargs={"db": db, "collection": collection}
        )
    )


In [ ]:
with shared.stub.run():
    # pull only lectures
    query = {"metadata.source": {"$regex": "youtube", "$options": "i"}}
    # project out the text field, it can get large
    projection = {"text": 0}
    # get just one result to show it worked
    result = shared.query_one.remote(query, projection, db=db, collection=collection)

pp.pprint(result)
